# Data Exploration
- No missing data in original data. Lots of missing data in IMDB set
- There are "rare" movies at the tail
    - 17% of movies have only 1 rating
    - 43% of movies have 5 or less ratings 
- Merge loses 17410 movies that aren't in the IMDB dataset

In [1]:
import pandas as pd
import os
import numpy as np
from scipy.sparse import csc_matrix
from pandas.api.types import CategoricalDtype

In [2]:
links = pd.read_csv('data/ml-25m/links.csv')
movies = pd.read_csv('data/ml-25m/movies.csv')
ratings = pd.read_csv('data/ml-25m/ratings.csv')

In [3]:
# distribution of number of ratings per movie. Many don't have many ratings. 
x = ratings.groupby('movieId').count()
x.userId.describe()

count    59047.000000
mean       423.393144
std       2477.885821
min          1.000000
25%          2.000000
50%          6.000000
75%         36.000000
max      81491.000000
Name: userId, dtype: float64

In [4]:
len(x[x.userId == 1]) / len(x)

0.174403441326401

In [5]:
len(x[x.userId <= 2]) / len(x)

0.3036733449624875

In [6]:
len(x[x.userId <= 5]) / len(x)

0.48842447541788747

### Merge all Data (including IMDB dataset)
Searched kaggle for a larger set and did not find one that also had IMDB IDs

In [7]:
# IMDB dataset
imdb_movies = pd.read_csv('data/imdb/IMDb movies.csv')

# standardize IMDB IDs
imdb_movies['imdbId'] = imdb_movies.imdb_title_id.str.split('tt').str[1]
imdb_movies.imdbId = pd.to_numeric(imdb_movies.imdbId)

In [8]:
# merge all dataset together 
num_movies = len(movies)
# links (has imdb rating) + movies
df = pd.merge(links, movies, on = 'movieId')
# merge with imdb data
df = pd.merge(df, imdb_movies, on = 'imdbId')
# titles are different in movielens vs imdb because imdb in the original language whereas movielens all english translated
df = df.rename(columns = {'title_x':'title_eng', 'title_y':'title_orig'})

new_num_movies = len(df)

## Missing Data Post Merge

In [9]:
# number of movies lost in merge
num_movies - new_num_movies

17410

In [10]:
new_num_movies

45013

In [11]:
df.isnull().sum()

movieId                      0
imdbId                       0
tmdbId                      11
title_eng                    0
genres                       0
imdb_title_id                0
title_orig                   0
original_title               0
year                         0
date_published               0
genre                        0
duration                     0
country                     10
language                   318
director                    13
writer                     348
production_company        1230
actors                      20
description                404
avg_vote                     0
votes                        0
budget                   29122
usa_gross_income         31246
worlwide_gross_income    23695
metascore                33044
reviews_from_users        1383
reviews_from_critics      2152
dtype: int64

## Separate Genres -> Dummy Variables
IMDB and MovieLens sometimes have a differnet genre list for thes same movie. Take the union of both lists to get the max number of genres/information.  

In [12]:
# convert into sets and take union 
df.genre = df.genre.str.split(', ')
df.genres = df.genres.str.split('|')

df.genre = df.genre.apply(set)
df.genres = df.genres.apply(set)

df['genres_all'] = df.apply(lambda x: x['genre'].union(x['genres']), axis=1)
df.genres_all = df.genres_all.apply(list)

In [13]:
# dummy variables
genre_dummies = pd.get_dummies(df.genres_all.apply(pd.Series).stack(), prefix = 'genre').sum(level = 0)

# merge back in
df = pd.merge(df, genre_dummies, left_index = True, right_index = True)

## Create User-Ratings Utility Matrix

In [24]:
ratings = ratings.sort_values(['userId', 'movieId'])
users_c = CategoricalDtype(sorted(ratings.userId.unique()), ordered=True)
movies_c = CategoricalDtype(sorted(ratings.movieId.unique()), ordered=True)

row = ratings.userId.astype(users_c).cat.codes
col = ratings.movieId.astype(movies_c).cat.codes
utility_matrix = csc_matrix((ratings.rating, (row, col)), 
                           shape=(users_c.categories.size, movies_c.categories.size))

utility_matrix.shape # row = users, columns = movies

In [72]:
# check: look at userId = 1, what movies have non-zero values in the matrix?
print(utility_matrix[0,:].nonzero())

# find indexes of movie IDs 
movies_lst = sorted(ratings.movieId.unique())
movies_lst.index(296) # movieId 296 = index 292. First nonzero element in sparse matrix. 

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0]), array([ 292,  302,  303,  654,  878, 1061, 1147, 1186, 1205, 1217, 1227,
       1591, 1922, 1923, 1979, 2071, 2260, 2482, 2541, 2600, 2751, 3352,
       3470, 3845, 4040, 4203, 4220, 4317, 4598, 4867, 5041, 5161, 5573,
       5655, 5766, 5800, 5840, 5904, 6252, 6258, 6416, 6588, 6829, 7084,
       7109, 7193, 7198, 7202, 7236, 7240, 7476, 7547, 7548, 7549, 7550,
       7593, 7653, 7720, 7734, 7775, 7972, 8015, 8072, 8158, 8258, 9120,
       9143, 9304, 9645, 9778]))


292

In [73]:
ratings[ratings.userId == 1]

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
65,1,27193,3.0,1147879774
66,1,27266,4.5,1147879365
67,1,27721,3.0,1147869115
68,1,31956,3.5,1147877610
